In [1]:
import numpy as np
import pandas as pd

%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [71]:
filename = 'smsspamcollection/SMSSpamCollection.txt'
with open(filename) as f:
    data = f.readlines()

In [72]:
y = [0 if data[i].split('\t')[0] == 'ham' else 1 for i in xrange(len(data))]
text = [data[i].split('\t')[1] for i in xrange(len(data))]

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(text)

## 5. Оценим качество

In [77]:
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
#X_train, X_test, y_train, y_test = train_test_split(sparse_data, label)
clf = LogisticRegression()
print np.mean(cross_val_score(clf, X, y, cv=10, scoring='f1'))

0.933348526858


## 6.Обучим и спрогнозируем

In [61]:
test_text = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", 
"FreeMsg: Txt: claim your reward of 3 hours talk time", 
"Have you visited the last lecture on physics?", 
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]

In [62]:
X_test = cv.transform(test_text)
#X_test = transformer.transform(X_test_sparse)

In [63]:
clf.fit(X, y)
y_pred = clf.predict(X_test)

In [66]:
print(y_pred)

[1 1 0 0 0]


## 7-8. Поэкспериментируем

In [67]:
ngrams=[(2,2), (3,3), (1,3)]
for range in ngrams:
    cv = CountVectorizer(ngram_range=range)
    X = cv.fit_transform(text)
    clf = LogisticRegression()
    print np.mean(cross_val_score(clf, X, y, cv=10, scoring='f1'))

0.822422066419
0.725016155547
0.925138255865


In [68]:
from sklearn.naive_bayes import MultinomialNB
ngrams=[(2,2), (3,3), (1,3)]
for range in ngrams:
    cv = CountVectorizer(ngram_range=range)
    X = cv.fit_transform(text)
    clf = MultinomialNB()
    print np.mean(cross_val_score(clf, X, y, cv=10, scoring='f1'))

0.645455401356
0.378623430876
0.887905460889


## 9. TF-IDF

In [74]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
X_tfidf = transformer.fit_transform(X)

In [76]:
clf = LogisticRegression()
print np.mean(cross_val_score(clf, X_tfidf, y, cv=10, scoring='f1'))

0.852859955417


Как мы можем видеть, на данном датасете качество понизилось

## 10. Мои решения

In [86]:
cv = CountVectorizer()
X = cv.fit_transform(text)

In [106]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
print np.mean(cross_val_score(clf, X, y, cv=10, scoring='f1'))

0.935910096434


In [92]:
from sklearn.grid_search import GridSearchCV
param_test = {'alpha':(1e-2, 1e-3, 1e-4)}
gsearch = GridSearchCV(estimator = SGDClassifier(loss='hinge',), param_grid = param_test, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch.fit(X, y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.91136, std: 0.01391, params: {'alpha': 0.01},
  mean: 0.93535, std: 0.01146, params: {'alpha': 0.001},
  mean: 0.93321, std: 0.00941, params: {'alpha': 0.0001}],
 {'alpha': 0.001},
 0.93535169820194175)

In [95]:
from sklearn.grid_search import GridSearchCV
param_test = {'n_iter':(1, 2, 3)}
gsearch = GridSearchCV(estimator = SGDClassifier(loss='hinge', alpha=1e-3), param_grid = param_test, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch.fit(X, y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.92746, std: 0.00951, params: {'n_iter': 1},
  mean: 0.93172, std: 0.00632, params: {'n_iter': 2},
  mean: 0.92866, std: 0.00673, params: {'n_iter': 3}],
 {'n_iter': 2},
 0.93171534636820197)

In [98]:
from sklearn.grid_search import GridSearchCV
param_test = {'random_state':(10, 22, 42, 50)}
gsearch = GridSearchCV(estimator = SGDClassifier(loss='hinge', alpha=1e-3, n_iter=2), param_grid = param_test, scoring='f1',n_jobs=4,iid=False, cv=5)
gsearch.fit(X, y)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

([mean: 0.93424, std: 0.01099, params: {'random_state': 10},
  mean: 0.93394, std: 0.01544, params: {'random_state': 22},
  mean: 0.93476, std: 0.01737, params: {'random_state': 42},
  mean: 0.93153, std: 0.00742, params: {'random_state': 50}],
 {'random_state': 42},
 0.93475902680707357)

In [104]:
clf = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
print np.mean(cross_val_score(clf, X, y, cv=10, scoring='f1'))

0.939946652397


In [105]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor()
print np.mean(cross_val_score(clf, X, y, cv=10, scoring='f1'))

0.884550677467


## 11. Выводы

1. хорошо работает на SDGClassifier
2. TF-IDF на данном датасете работает так себе
3. Классификатор с учетом n-грамм работает хуже